# Bài 1: Sử dụng Python, thu thập lại dữ liệu về số ca nhiễm Corona ở từng quốc gia tại trang Worldometer: https://www.worldometers.info/coronavirus/#countries.

## Thu thập dữ liệu và lưu vào file csv

In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [19]:
url="https://www.worldometers.info/coronavirus/#countries." # Tạo url
response = requests.get(url) #gủi GET request đến url

In [20]:
# Check thành công
if response.status_code == 200:
    html_content = response.text
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

In [21]:
soup = BeautifulSoup(html_content, "html.parser") #parse html content

In [22]:
table=soup.find("table", attrs={"id":"main_table_countries_today"}) #tìm bảng chứa thông tin về số ca nhiễm

In [23]:
data=[]
columns=[]
for tr in table.find_all("tr"): #duyệt qua từng dòng trong bảng
    if('row_continent' not in (tr.get_attribute_list("class"))):
        section=[]
        for th in tr.find_all(["th"]): #head thì lưu vào columns
            columns.append(th.text.strip().replace("\n",""))
        for td in tr.find_all(["td"]): #các dữ liệu còn lại lưu vào data
            try:
                section.append(float(td.text.strip().replace(",","").replace("+","")))
            except:
                section.append(td.text.strip().replace(",","").replace("+",""))
        data.append(section)

In [24]:
data=pd.DataFrame(data[1:], columns=columns) #tạo dataframe từ dữ liệu lấy được
data

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,World,704607102.0,621.0,7009335.0,2.0,675433749.0,879.0,22164018.0,34933.0,...,,,,All,,,,,,
1,1.0,USA,111769834.0,,1218856.0,,109712052.0,,838926.0,1058.0,...,1186851502.0,3544901.0,334805269.0,North America,3.0,275.0,0.0,,,2506.0
2,2.0,India,45034136.0,,533547.0,,N/A,N/A,N/A,N/A,...,935879495.0,665334.0,1406631776.0,Asia,31.0,2636.0,2.0,,,0.6
3,3.0,France,40138560.0,,167642.0,,39970918.0,,0.0,,...,271490188.0,4139547.0,65584518.0,Europe,2.0,391.0,0.0,,,
4,4.0,Germany,38827040.0,,182917.0,,38240600.0,,403523.0,N/A,...,122332384.0,1458359.0,83883596.0,Europe,2.0,459.0,1.0,,,4811.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,228.0,Vatican City,29.0,,,,29.0,,0.0,,...,,,799.0,Europe,28.0,,,,,
229,229.0,Western Sahara,10.0,,1.0,,9.0,,0.0,,...,,,626161.0,Africa,62616.0,626161.0,,,,
230,230.0,MS Zaandam,9.0,,2.0,,7.0,,0.0,,...,,,,,,,,,,
231,231.0,China,503302.0,,5272.0,,379053.0,,118977.0,N/A,...,160000000.0,110461.0,1448471400.0,Asia,2878.0,274748.0,9.0,,,82.0


In [25]:
data.to_csv("covid19.csv", index=False) #lưu dữ liệu vào file csv

## Đọc dữ liệu từ file csv vừa thu thập và thực hiện các yêu cầu

In [26]:
df=pd.read_csv("covid19.csv",usecols=range(1,len(columns))) #đọc dữ liệu từ file csv
df=df.iloc[1:-1] #bỏ dòng đầu tiên và cuối vì chứa thông tin về thế giới
df.head() #hiển thị 5 dòng đầu tiên

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
1,USA,111769834.0,NaN,1218856.0,NaN,109712052.0,NaN,838926.0,1058.0,333835.0,...,1.186852e+09,3544901.0,3.348053e+08,North America,3.0,275.0,0.0,NaN,NaN,2506.0
2,India,45034136.0,NaN,533547.0,NaN,NaN,NaN,NaN,NaN,32016.0,...,9.358795e+08,665334.0,1.406632e+09,Asia,31.0,2636.0,2.0,NaN,NaN,0.6
3,France,40138560.0,NaN,167642.0,NaN,39970918.0,NaN,0.0,NaN,612013.0,...,2.714902e+08,4139547.0,6.558452e+07,Europe,2.0,391.0,0.0,NaN,NaN,NaN
4,Germany,38827040.0,NaN,182917.0,NaN,38240600.0,NaN,403523.0,NaN,462868.0,...,1.223324e+08,1458359.0,8.388360e+07,Europe,2.0,459.0,1.0,NaN,NaN,4811.0
5,Brazil,38729836.0,NaN,711249.0,NaN,36249161.0,NaN,1769426.0,NaN,179843.0,...,6.377617e+07,296146.0,2.153536e+08,South America,6.0,303.0,3.0,NaN,NaN,8216.0


In [27]:
df.info() #hiển thị thông tin của dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 1 to 231
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country,Other        231 non-null    object 
 1   TotalCases           231 non-null    float64
 2   NewCases             1 non-null      float64
 3   TotalDeaths          226 non-null    float64
 4   NewDeaths            1 non-null      float64
 5   TotalRecovered       183 non-null    float64
 6   NewRecovered         6 non-null      float64
 7   ActiveCases          184 non-null    float64
 8   Serious,Critical     52 non-null     float64
 9   Tot Cases/1M pop     229 non-null    float64
 10  Deaths/1M pop        224 non-null    float64
 11  TotalTests           213 non-null    float64
 12  Tests/1M pop         213 non-null    float64
 13  Population           229 non-null    float64
 14  Continent            229 non-null    object 
 15  1 Caseevery X ppl    229 non-null    flo

### a) Tìm 5 quốc gia có số ca nhiễm (Total case) nhiều nhất.

In [28]:
df.sort_values(by="TotalCases", ascending=False)[['Country,Other','TotalCases']].head(5) #sắp xếp theo số ca nhiễm giảm dần và hiển thị 5 quốc gia có số ca nhiễm nhiều nhất

,"Country,Other",TotalCases
1,USA,111769834.0
2,India,45034136.0
3,France,40138560.0
4,Germany,38827040.0
5,Brazil,38729836.0


### b) Quốc gia nào có số ca nhiễm mới cao nhất?

In [29]:
df[df['NewCases']==df['NewCases'].max()][['Country,Other','NewCases']]

,"Country,Other",NewCases
13,Australia,621.0


### c) Tính tỉ lệ tổng số ca bình phục trên tổng số ca nhiễm. Xác định 3 quốc gia có tỉ lệ bình phục cao nhất. 

**Thêm cột Recovery rate là tỷ lệ bình phục**

In [30]:
df["Recovery rate"]=df["TotalRecovered"]/df["TotalCases"] #tính tỉ lệ phục hồi

**3 Quốc gia có tỉ lệ bình phục cao nhất**

In [31]:
df.sort_values(by="Recovery rate", ascending=False)[['Country,Other','Recovery rate']].head(3) #sắp xếp theo tỉ lệ phục hồi giảm dần và hiển thị 3 quốc gia có tỉ lệ phục hồi cao nhất

,"Country,Other",Recovery rate
223,Falkland Islands,1.000000
228,Vatican City,1.000000
32,DPRK,0.999984
